# viz-1b-flights-splom.ipynb

This notebook builds on the learnings from [the first one](ttps://walterra.github.io/jupyter2kibana/viz-1a-flights-histogram.html) and uses the same data to create and deploy a scatterplot matrix. The difference here is that we moved the code to create the SavedObject in Kibana to a helper function into [kibana_vega_util.py](https://github.com/walterra/jupyter2kibana/blob/master/kibana_vega_util.py) so we can reuse it.

In [1]:
import datetime
import altair as alt
import eland as ed
import json
import numpy as np
import matplotlib.pyplot as plt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
url = 'http://localhost:9200/kibana_sample_data_flights/_search?size=1000'
url_data = alt.Data(url=url, format=alt.DataFormat(property='hits.hits',type='json'))

fields = [
    'Carrier',
    'AvgTicketPrice',
    'DistanceKilometers',
    'DistanceMiles',
    'FlightDelayMin',
    'FlightTimeMin',
    'dayOfWeek'
]

rename_dict = dict((a, 'datum._source.'+a) for a in fields)

chart = alt.Chart(url_data).mark_circle(
    opacity=.5,
    size=6
).transform_calculate(**rename_dict).encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative')
).properties(
    width=100,
    height=100
).repeat(
    row=['AvgTicketPrice', 'DistanceKilometers', 'FlightDelayMin', 'FlightTimeMin'],
    column=['FlightTimeMin', 'FlightDelayMin', 'DistanceKilometers', 'AvgTicketPrice']
).interactive()

chart

alt.RepeatChart(...)

In [3]:
from save_vega_vis import saveVegaVis
from elasticsearch import Elasticsearch 
es=Elasticsearch([{'host':'localhost','port':9200}])

saveVegaVis(es, 'kibana_sample_data_flights', 'def-vega-splom-1', chart, resultSize=1000)

ModuleNotFoundError: No module named 'save_vega_vis'